In [ ]:
import pandas as pd
from fuzzywuzzy import process
from collections import defaultdict

In [ ]:
file_path = '/mnt/data/all_ingredients.csv'
ingredients_df = pd.read_csv(file_path)
ingredients_df.dropna(inplace=True)

ingredients_df.head()

Para tratar as variações dos ingredientes, como "abacate médio" e "abacate tipo avocado", iremos agrupa-las sob um termo único como "abacate".

Para isso, podemos utilizar o algoritmo de distância de Levenshtein para identificar ingredientes semelhantes e decidir um nome padrão. Vamos começar por listar as variações mais comuns para cada grupo principal de ingredientes:

In [ ]:
# Agrupar ingredientes semelhantes usando a distância de Levenshtein
def group_similar_ingredients(ingredient_list, similarity_threshold=85):
    # Dicionário para armazenar os grupos de ingredientes
    ingredient_groups = defaultdict(list)
    
    # Lista já processada
    processed = set()
    
    # Agrupar ingredientes semelhantes
    for ingredient in ingredient_list:
        if ingredient in processed:
            continue
        # Encontrar todos os ingredientes semelhantes
        matches = process.extract(ingredient, ingredient_list, limit=None, scorer=fuzz.token_sort_ratio)
        similar = [match[0] for match in matches if match[1] >= similarity_threshold]
        # Adicionar ao dicionário de grupos
        ingredient_groups[ingredient].extend(similar)
        # Marcar como processados
        processed.update(similar)
        
    return ingredient_groups

# Agrupar ingredientes
ingredient_list = ingredients_df['Ingrediente'].unique()
ingredient_groups = group_similar_ingredients(ingredient_list)

# Mostrar alguns grupos de exemplo
example_groups = {k: v for k, v in list(ingredient_groups.items())[:10]}
example_groups


Parece que ocorreu um erro por não ter importado o módulo fuzz corretamente. Vou corrigir isso e executar novamente o código para agrupar os ingredientes semelhantes. Vamos tentar novamente: 

In [ ]:
from fuzzywuzzy import fuzz

# Corrigir a função de agrupamento com a importação correta e executar novamente
ingredient_groups = group_similar_ingredients(ingredient_list)

# Mostrar alguns grupos de exemplo
example_groups = {k: v for k, v in list(ingredient_groups.items())[:10]}
example_groups


O processo de agrupamento de ingredientes semelhantes levou mais tempo do que o esperado e acabou sendo interrompido. Vamos tentar uma abordagem diferente que possa ser mais eficiente.

Uma possível solução é reduzir o número de comparações usando uma pré-seleção dos ingredientes mais comuns e depois agrupar apenas os menos comuns em torno desses centros. Isso pode ser feito utilizando uma análise de frequência para identificar os ingredientes mais comuns e agrupar os similares a estes.

In [ ]:
# Calcular a frequência de cada ingrediente
ingredient_counts = ingredients_df['Ingrediente'].value_counts()

# Selecionar os ingredientes mais comuns como centros de agrupamento
common_ingredients = ingredient_counts[ingredient_counts > 5].index.tolist()  # Ajustar o limiar conforme necessário

# Função para agrupar em torno de centros comuns
def group_around_commons(ingredient_list, common_centers, similarity_threshold=85):
    grouped_ingredients = defaultdict(list)
    
    # Agrupar em torno dos centros comuns
    for ingredient in ingredient_list:
        # Encontrar o centro comum mais similar
        match = process.extractOne(ingredient, common_centers, scorer=fuzz.token_sort_ratio)
        if match and match[1] >= similarity_threshold:
            grouped_ingredients[match[0]].append(ingredient)
        else:
            # Se nenhum centro comum for suficientemente similar, manter o ingrediente sozinho
            grouped_ingredients[ingredient].append(ingredient)
            
    return grouped_ingredients

# Aplicar a função de agrupamento
grouped_ingredients = group_around_commons(ingredient_list, common_ingredients)

# Mostrar alguns grupos de exemplo
example_grouped_ingredients = {k: v for k, v in list(grouped_ingredients.items())[:10]}
example_grouped_ingredients


Consegui agrupar alguns ingredientes em torno dos centros comuns, mas parece que ainda precisamos ajustar o limiar de similaridade para conseguir agrupamentos mais eficazes. Atualmente, muitos ingredientes permanecem sozinhos mesmo quando poderiam ser agrupados sob um termo mais geral.

Podemos ajustar o limiar de similaridade para tentar agrupar mais ingredientes ou considerar uma abordagem mais sofisticada, como técnicas de processamento (...)?